## General Board Control
[Quick reference for the ESP32](https://docs.micropython.org/en/latest/esp32/quickref.html#quick-reference-for-the-esp32)

![esp32s](images/esp32s.jpg)

In [2]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
I (454) cpu_start: Pro cpu up.
MicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [6]:
import machine
pin12 = machine.Pin(12, machine.Pin.OUT)
pin12.value(1)
pin13 = machine.Pin(13, machine.Pin.IN, machine.Pin.PULL_UP)
print(pin13.value())
i2c = machine.I2C(scl=machine.Pin(21), sda=machine.Pin(22))
i2c.scan()

1


In [7]:
import esp32
T = esp32.raw_temperature() # read the internal temperature of the MCU, in Farenheit
print(T)

125


## Delay and timing
#### Use the time module:

In [10]:
import time

time.sleep(1)           # sleep for 1 second
time.sleep_ms(500)      # sleep for 500 milliseconds
time.sleep_us(10)       # sleep for 10 microseconds
start = time.ticks_ms() # get millisecond counter
for i in range(1000):
    time.sleep_ms(1)
delta = time.ticks_diff(time.ticks_ms(), start) # compute time difference
print(start)
print(delta)

.592354
1034


## Pins and GPIO
#### Use the machine.Pin class:

#### 可用的引腳來自以下範圍（包括以下範圍）：0-19、21-23、25-27、32-39。這些對應於ESP32芯片的實際GPIO引腳號。請注意，許多最終用戶板使用自己的臨時引腳編號（例如，標記為D0，D1等）。有關板邏輯引腳和物理芯片引腳之間的映射，請查閱板文檔。

#### 筆記：

#### 引腳1和3分別是REPL UART TX和RX
#### 引腳6、7、8、11、16和17用於連接嵌入式閃存，不建議用於其他用途
#### 引腳34-39僅用於輸入，也沒有內部上拉電阻
#### 可以設置某些引腳的上拉值Pin.PULL_HOLD以降低深度睡眠期間的功耗。

In [11]:
from machine import Pin

p0 = Pin(0, Pin.OUT)    # create output pin on GPIO0
p0.on()                 # set pin to "on" (high) level
p0.off()                # set pin to "off" (low) level
p0.value(1)             # set pin to on/high

p2 = Pin(2, Pin.IN)     # create input pin on GPIO2
print(p2.value())       # get value, 0 or 1

p4 = Pin(4, Pin.IN, Pin.PULL_UP) # enable internal pull-up resistor
p5 = Pin(5, Pin.OUT, value=1) # set pin high on creation

0


## PWM（脈衝寬度調製）
#### 可以在所有使能輸出的引腳上使能PWM。
#### 基本頻率可以在1Hz至40MHz的範圍內，但需要權衡取捨。
#### 隨著基頻的 增加，佔空比分辨率降低。
###### 有關 更多詳細信息，請參見 LED控制。

In [ ]:
from machine import Pin, PWM

pwm0 = PWM(Pin(0))      # 從1個引腳中創建PWM對象
pwm0.freq()             # 獲取當前頻率
pwm0.freq(1000)         # 設置頻率
pwm0.duty()             # 獲取當前占空比
pwm0.duty(200)          # 設置占空比
pwm0.deinit()           # 關閉引腳的 PWM

pwm2 = PWM(Pin(2), freq=20000, duty=512) # 在同一語句下創建和配置 PWM

## ADC (類比數位轉換)
#### ADC功能在ESP32引腳32-39上可用。請注意，使用預設配置時，ADC引腳上的輸入電壓必須 介於0.0v和1.0v之間（任何高於1.0v的值#### 都將讀為4095）。如果需要增加測量範圍，需要配置 降壓電路。

## ESP32 特定的 ADC class使用方法說明:
### ADC.atten(attenuation)
#### 該方法允許設置ADC輸入的衰減量，以獲取更大的電壓測量範圍，但是以精度為代價的。 （配置後相同的位元數表示更寬的範圍）。衰減選項如下:
*	ADC.ATTN_0DB: 0dB 衰減, 最大輸入電壓為 1.00v - 這是預設配置
*	ADC.ATTN_2_5DB: 2.5dB 衰減, 最大輸入電壓約為 1.34v
*	ADC.ATTN_6DB: 6dB 衰減, 最大輸入電壓約為 2.00v
*	ADC.ATTN_11DB: 11dB 衰減, 最大輸入電壓約為3v

## 警告
### 儘管通過配置11dB衰減可以讓測量電壓到達3.6v,
### 但由於ESP32晶片的最大允許輸入電壓是3.6V, 
### 因此輸入接近3.6V的電壓可能會導致IC燒壞！

In [ ]:
from machine import ADC

adc = ADC(Pin(32))          # 在ADC引腳上創建ADC物件
adc.read()                  # 讀取測量值, 0-4095 表示電壓從 0.0v - 1.0v

adc.atten(ADC.ATTN_11DB)    # 設置 11dB 衰減輸入 (測量電壓大致從 0.0v - 3.6v)
adc.width(ADC.WIDTH_9BIT)   # 設置 9位 精度輸出 (返回值 0-511)
adc.read()                  # 獲取重新配置後的測量值

In [12]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x1fd194de908, open=True>(port='COM4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
